In [2]:
import torch
import numpy as np
from wisent.core.models.wisent_model import WisentModel

# Configuration
MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"

# Load model and tokenizer using WisentModel for consistent settings
print(f"Loading {MODEL_NAME}...")
wisent_model = WisentModel(model_name=MODEL_NAME)
model = wisent_model.hf_model
tokenizer = wisent_model.tokenizer

print(f"Model loaded!")
print(f"Number of layers: {model.config.num_hidden_layers}")
print(f"Hidden dimension: {model.config.hidden_size}")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading meta-llama/Llama-3.2-1B-Instruct...
Model loaded!
Number of layers: 16
Hidden dimension: 2048


### Visualizing Activation Shape

Let's pass a simple prompt through the model and examine the activation tensors.

In [3]:
# A simple prompt
prompt = "The capital of France is"

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
print(f"Input prompt: '{prompt}'")
print(f"Token IDs: {inputs.input_ids[0].tolist()}")
print(f"Tokens: {[tokenizer.decode([t]) for t in inputs.input_ids[0]]}")
print(f"Number of tokens: {inputs.input_ids.shape[1]}")

Input prompt: 'The capital of France is'
Token IDs: [128000, 791, 6864, 315, 9822, 374]
Tokens: ['<|begin_of_text|>', 'The', ' capital', ' of', ' France', ' is']
Number of tokens: 6


In [4]:
# Forward pass to get hidden states
with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True)

# hidden_states is a tuple: (embedding_output, layer_1, layer_2, ..., layer_n)
hidden_states = outputs.hidden_states

print(f"Number of hidden state tensors: {len(hidden_states)}")
print(f"(This is 1 embedding layer + {len(hidden_states)-1} transformer layers)")
print()

# Examine a specific layer's activations
layer_idx = 8  # Middle layer
layer_activations = hidden_states[layer_idx]

print(f"Layer {layer_idx} activations:")
print(f"  Shape: {layer_activations.shape}")
print(f"  - Batch size: {layer_activations.shape[0]}")
print(f"  - Sequence length: {layer_activations.shape[1]}")
print(f"  - Hidden dimension: {layer_activations.shape[2]}")

Number of hidden state tensors: 17
(This is 1 embedding layer + 16 transformer layers)

Layer 8 activations:
  Shape: torch.Size([1, 6, 2048])
  - Batch size: 1
  - Sequence length: 6
  - Hidden dimension: 2048


In representation engineering, we often care about activations at specific positions:

- **Last token**: The most common choice - represents the "state" of the model after processing the full context
- **Mean pooling**: Average across all tokens for a more holistic representation
- **Specific positions**: Sometimes we want activations at particular token positions

### Getting the Last Token Activation

In [5]:
# Get activation at the last token position for layer 8
layer_idx = 8
last_token_idx = -1  # Last position

# Extract: [batch, seq, hidden] -> [hidden] for last token
last_token_activation = hidden_states[layer_idx][0, last_token_idx, :]

print(f"Last token activation for layer {layer_idx}:")
print(f"  Shape: {last_token_activation.shape}")
print(f"  This is a {last_token_activation.shape[0]}-dimensional vector!")
print()
print(f"  First 10 values: {last_token_activation[:10].tolist()}")
print(f"  L2 Norm: {torch.norm(last_token_activation).item():.4f}")
print(f"  Mean: {last_token_activation.mean().item():.4f}")
print(f"  Std: {last_token_activation.std().item():.4f}")

Last token activation for layer 8:
  Shape: torch.Size([2048])
  This is a 2048-dimensional vector!

  First 10 values: [0.025029459968209267, -0.08347947895526886, -0.09321831911802292, -0.001393824815750122, -0.1272227019071579, -0.0920216366648674, 0.010746791958808899, -0.12839482724666595, 0.03147208318114281, 0.0033951252698898315]
  L2 Norm: 5.5896
  Mean: 0.0032
  Std: 0.1235


### Mean Pooling Across Tokens

In [6]:
# Mean pooling: average across all token positions
mean_activation = hidden_states[layer_idx][0].mean(dim=0)  # [seq, hidden] -> [hidden]

print(f"Mean-pooled activation for layer {layer_idx}:")
print(f"  Shape: {mean_activation.shape}")
print(f"  L2 Norm: {torch.norm(mean_activation).item():.4f}")

# Compare with last token
cosine_sim = torch.nn.functional.cosine_similarity(
    last_token_activation.unsqueeze(0),
    mean_activation.unsqueeze(0)
).item()
print(f"\nCosine similarity between last-token and mean-pooled: {cosine_sim:.4f}")

Mean-pooled activation for layer 8:
  Shape: torch.Size([2048])
  L2 Norm: 132.4203

Cosine similarity between last-token and mean-pooled: 0.0050


In [7]:
from wisent.core.activations.core.atoms import LayerActivations, ActivationAggregationStrategy

# Create a LayerActivations object manually
# In practice, wisent's ActivationsCollector does this for you
layer_activations_dict = {
    f"layer_{i}": hidden_states[i][0, -1, :]  # Last token for each layer
    for i in range(1, len(hidden_states))  # Skip embedding layer
}

activations = LayerActivations(layer_activations_dict)

print(f"LayerActivations object:")
print(f"  Number of layers: {len(activations)}")
print(f"  Layer names: {list(activations.keys())[:5]}... (showing first 5)")
print(f"  Layer 8 shape: {activations['layer_8'].shape}")

LayerActivations object:
  Number of layers: 16
  Layer names: ['layer_1', 'layer_2', 'layer_3', 'layer_4', 'layer_5']... (showing first 5)
  Layer 8 shape: torch.Size([2048])


In [8]:
import os

OUTPUT_DIR = "./steering_outputs"
LAYER = 8

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/vectors", exist_ok=True)

# Generate a "happy" steering vector using CLI
# This will:
# 1. Generate synthetic positive (happy) and negative (sad) response pairs
# 2. Collect activations from both
# 3. Compute steering_vector = mean(positive) - mean(negative)

HAPPY_TRAIT = "happy, joyful, optimistic, cheerful responses full of positivity and enthusiasm"

!python -m wisent.core.main generate-vector-from-synthetic \
    --trait "{HAPPY_TRAIT}" \
    --model {MODEL_NAME} \
    --num-pairs 10 \
    --layers {LAYER} \
    --output {OUTPUT_DIR}/vectors/happy_vector.pt \
    --normalize \
    --verbose

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



🎯 Generating Steering Vector from Synthetic Pairs (Full Pipeline)
   Trait: happy, joyful, optimistic, cheerful responses full of positivity and enthusiasm
   Model: meta-llama/Llama-3.2-1B-Instruct
   Num Pairs: 10

Step 1/3: Generating synthetic contrastive pairs...


🎨 Generating synthetic contrastive pairs
   Trait: happy, joyful, optimistic, cheerful responses full of positivity and enthusiasm
   Number of pairs: 10
   Model: meta-llama/Llama-3.2-1B-Instruct

🤖 Loading model 'meta-llama/Llama-3.2-1B-Instruct'...
`torch_dtype` is deprecated! Use `dtype` instead!
   ✓ Model loaded with 16 layers

🧹 Setting up cleaning pipeline...

⚙️  Initializing generator...

🎯 Generating 10 contrastive pairs...
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
   ✓ Generated 10 pairs

📊 Generation Report:
   Requeste

In [10]:
# Load the generated steering vector
steering_data = torch.load(f"{OUTPUT_DIR}/vectors/happy_vector.pt")
steering_vector_normalized = steering_data["vector"]

print(f"Loaded steering vector:")
print(f"  Shape: {steering_vector_normalized.shape}")
print(f"  L2 norm: {torch.norm(steering_vector_normalized).item():.4f}")
print(f"  First 10 values: {steering_vector_normalized[:10].tolist()}")

Loaded steering vector:
  Shape: torch.Size([2048])
  L2 norm: 1.0000
  First 10 values: [0.01185621414333582, -0.014781808480620384, 0.002184569602832198, -0.01078332681208849, -0.004495644476264715, 0.024912234395742416, 0.022735361009836197, 0.02050434984266758, 0.01592581532895565, 0.0009902934543788433]


In [11]:
class SteeringHook:
    """A hook that adds a steering vector to activations."""
    
    def __init__(self, steering_vector, alpha=1.0):
        self.steering_vector = steering_vector
        self.alpha = alpha
        self.handle = None
    
    def __call__(self, module, input, output):
        """Called during forward pass. Modifies the output activation."""
        # output is typically (hidden_states, ...)
        if isinstance(output, tuple):
            hidden_states = output[0]
        else:
            hidden_states = output
        
        # Add steering vector (broadcast across batch and sequence)
        # steering_vector: [hidden_dim] -> [1, 1, hidden_dim]
        sv = self.steering_vector.to(hidden_states.device).to(hidden_states.dtype)
        sv = sv.unsqueeze(0).unsqueeze(0)
        
        modified = hidden_states + self.alpha * sv
        
        if isinstance(output, tuple):
            return (modified,) + output[1:]
        return modified
    
    def attach(self, layer):
        """Attach the hook to a layer."""
        self.handle = layer.register_forward_hook(self)
    
    def remove(self):
        """Remove the hook."""
        if self.handle:
            self.handle.remove()

print("SteeringHook class defined!")
print("This modifies activations during the forward pass.")

SteeringHook class defined!
This modifies activations during the forward pass.


In [12]:
# Baseline (no steering)
print("=" * 60)
print("BASELINE (No Steering)")
print("=" * 60)

!python -m wisent.core.main multi-steer \
    --model {MODEL_NAME} \
    --prompt "Today I woke up and felt" \
    --max-new-tokens 50

BASELINE (No Steering)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



🎯 Unsteered Baseline Generation Mode
   Model: meta-llama/Llama-3.2-1B-Instruct
   No steering vectors provided - generating baseline response

🤖 Loading model 'meta-llama/Llama-3.2-1B-Instruct'...
`torch_dtype` is deprecated! Use `dtype` instead!
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.

Unsteered baseline output:
It seems like you started your day with a thought. You mentioned feeling something, but didn't specify what it was. Would you like to share more about how you're feeling now? Sometimes talking about our emotions can help clarify them.


✅ Baseline generation completed successfully!



In [13]:
import subprocess

# Positive steering (toward happy)
print("=" * 60)
print("POSITIVE STEERING (strength=1.5 -> happy)")
print("=" * 60)

result = subprocess.run(
    f'python -m wisent.core.main multi-steer '
    f'--vector {OUTPUT_DIR}/vectors/happy_vector.pt:1.5 '
    f'--model {MODEL_NAME} '
    f'--layer {LAYER} '
    f'--prompt "Today I woke up and felt" '
    f'--max-new-tokens 50',
    shell=True, capture_output=True, text=True
)
print(result.stdout)
happy_output = result.stdout

POSITIVE STEERING (strength=1.5 -> happy)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



🎯 Multi-Steering Mode
   Model: meta-llama/Llama-3.2-1B-Instruct
   Layer: 8
   Method: CAA
Loading vector from ./steering_outputs/vectors/happy_vector.pt with weight 1.5
   ✓ Loaded vector from layer 8

Using CAA method for vector combination
Target layer: 8

🔄 Combining 1 vectors
   ✓ Combined vector shape: torch.Size([2048])
   ✓ Combined vector norm: 1.0000
   ✓ Steering scale: 1.5000

🤖 Loading model 'meta-llama/Llama-3.2-1B-Instruct'...
`torch_dtype` is deprecated! Use `dtype` instead!

🎯 Applying combined steering vector at layer 8
Prompt: Today I woke up and felt
Prompt is formatted: False
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.

Generated output:
I'd love to help you explore your feelings. What's been going on lately that's made you feel something new? Is it a specific experience, emotio

In [14]:
# Negative steering (toward sad)
print("=" * 60)
print("NEGATIVE STEERING (strength=-1.5 -> sad)")
print("=" * 60)

result = subprocess.run(
    f'python -m wisent.core.main multi-steer '
    f'--vector {OUTPUT_DIR}/vectors/happy_vector.pt:-1.5 '
    f'--model {MODEL_NAME} '
    f'--layer {LAYER} '
    f'--prompt "Today I woke up and felt" '
    f'--max-new-tokens 50',
    shell=True, capture_output=True, text=True
)
print(result.stdout)
sad_output = result.stdout

NEGATIVE STEERING (strength=-1.5 -> sad)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



🎯 Multi-Steering Mode
   Model: meta-llama/Llama-3.2-1B-Instruct
   Layer: 8
   Method: CAA
Loading vector from ./steering_outputs/vectors/happy_vector.pt with weight -1.5
   ✓ Loaded vector from layer 8

Using CAA method for vector combination
Target layer: 8

🔄 Combining 1 vectors
   ✓ Combined vector shape: torch.Size([2048])
   ✓ Combined vector norm: 1.0000
   ✓ Steering scale: -1.5000

🤖 Loading model 'meta-llama/Llama-3.2-1B-Instruct'...
`torch_dtype` is deprecated! Use `dtype` instead!

🎯 Applying combined steering vector at layer 8
Prompt: Today I woke up and felt
Prompt is formatted: False
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.

Generated output:
I can provide general responses to help you complete your sentence. Would you like me to continue?


✅ Multi-steering completed successfully!

In [16]:
import re

def extract_generated_output(output):
    """Extract just the generated text from CLI output."""
    match = re.search(r'Generated output:\n(.+?)\n\n', output, re.DOTALL)
    if match:
        return match.group(1).strip()
    return output

print("=" * 60)
print("COMPARISON: Happy vs Sad Steering")
print("=" * 60)
print()
print("HAPPY (strength=1.5):")
print("-" * 40)
print(extract_generated_output(happy_output))
print()
print("SAD (strength=-1.5):")
print("-" * 40)
print(extract_generated_output(sad_output))

COMPARISON: Happy vs Sad Steering

HAPPY (strength=1.5):
----------------------------------------


NameError: name 'happy_output' is not defined

## Extracting Activations from a Single Pair

Let's see how to extract activations from one contrastive pair using the CLI.

In [17]:
import json
import os

# Create a single contrastive pair
pair = {
    "pairs": [
        {
            "prompt": "How are you feeling today?",
            "positive_response": {"model_response": "I'm feeling absolutely wonderful! Everything is going great!"},
            "negative_response": {"model_response": "I'm feeling terrible. Everything is going wrong."}
        }
    ],
    "num_pairs": 1
}

# Save the pair to a file
pairs_file = f"{OUTPUT_DIR}/single_pair.json"
with open(pairs_file, "w") as f:
    json.dump(pair, f, indent=2)

print(f"Saved pair to {pairs_file}")
print(f"Prompt: {pair['pairs'][0]['prompt']}")
print(f"Positive: {pair['pairs'][0]['positive_response']['model_response']}")
print(f"Negative: {pair['pairs'][0]['negative_response']['model_response']}")

Saved pair to ./steering_outputs/single_pair.json
Prompt: How are you feeling today?
Positive: I'm feeling absolutely wonderful! Everything is going great!
Negative: I'm feeling terrible. Everything is going wrong.


In [18]:
# Extract activations using CLI
enriched_file = f"{OUTPUT_DIR}/single_pair_enriched.json"

!python -m wisent.core.main get-activations \
    {pairs_file} \
    --model {MODEL_NAME} \
    --layers {LAYER} \
    --token-aggregation final \
    --output {enriched_file} \
    --verbose

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


usage: main.py [-h] [--verbose]
               {tasks,generate-pairs,diagnose-pairs,generate-pairs-from-task,get-activations,diagnose-vectors,create-steering-vector,generate-vector-from-task,generate-vector-from-synthetic,synthetic,test-nonsense,monitor,agent,model-config,configure-model,optimize-classification,optimize-steering,optimize-sample-size,optimize-all,optimize,generate-vector,multi-steer,evaluate,generate-responses,evaluate-responses,modify-weights,evaluate-refusal,inference-config,optimization-cache,optimize-weights,train-unified-goodness,check-linearity}
               ...
main.py: error: unrecognized arguments: --pairs-file


In [19]:
# Load and inspect the activations
with open(enriched_file, "r") as f:
    enriched = json.load(f)

pair_data = enriched["pairs"][0]
pos_activations = pair_data["positive_response"]["activations"]
neg_activations = pair_data["negative_response"]["activations"]

print("Positive response activations:")
print(f"  Layers: {list(pos_activations.keys())}")
print(f"  Layer {LAYER} shape: {len(pos_activations[str(LAYER)])} dimensions")
print(f"  First 10 values: {pos_activations[str(LAYER)][:10]}")

print("\nNegative response activations:")
print(f"  Layers: {list(neg_activations.keys())}")
print(f"  Layer {LAYER} shape: {len(neg_activations[str(LAYER)])} dimensions")
print(f"  First 10 values: {neg_activations[str(LAYER)][:10]}")

# Compute difference (this is what CAA uses)
import numpy as np
pos_vec = np.array(pos_activations[str(LAYER)])
neg_vec = np.array(neg_activations[str(LAYER)])
diff = pos_vec - neg_vec

print("\nDifference vector (positive - negative):")
print(f"  Shape: {diff.shape}")
print(f"  Norm: {np.linalg.norm(diff):.4f}")
print(f"  First 10 values: {diff[:10].tolist()}")

FileNotFoundError: [Errno 2] No such file or directory: './steering_outputs/single_pair_enriched.json'